# Bash on the Commandline
Date: 2020-01-14
Author: Jason Beach
Categories: DevOps, SystemAdmin
Tags: linux, bash, tag3
<!--eofm-->

Even the most stripped-down of linux machines will come with bash.  This comes in handy when working with minimal Docker containers, Vagrant VMs, or even really cheap customers who can't shell out money for a shell.  The following are a few fundamentals to jog your memory when you get stuck programming without the typical goodie-bags of an IDE or scipting language.

## Environment

### System

* `/etc` - Stores config files for the system.
* `/var/log` - Stores log files for various system programs. (You may not have permission to look at everything in this directory. Don't let that stop you exploring though. A few error messages never hurt anyone.)
* `/bin` - The location of several commonly used programs (some of which we will learn about in the rest of this tutorial.
* `/usr/bin` - Another location for programs on the system.

Use `file <path_to_file>` to determine what type of file it is (directory, file, empty)
`ls --all` long hand command line options begin with two dashes
`ls -a` short hand options begin with a single dash
`ls -alh` when we use a single dash we may invoke several options by placing all the letters representing those options together after the dash
`man -k <search term>` do a keyword search for all manual pages containing the given search term.

### Wildcards

* `*` - represents zero or more characters
* `?` - represents a single character
* `[]` - represents a range of characters

### Permissions

```
drwx-w-r-x
 UsrGrpOthr
 111010101
 7  2  5
 ```
 
* `chmod g+x frog.png` - group grant execute
* `chmod 755 frog.png` - using octal values, change to: `-rwxr-xr-x`, a common permission status

## Syntax

### Basic commands

```
let <arithmetic expression>
  let a=5+4
  let "a = 5 + 4"
  a=$( expr 5 + 4 )
  a=$(( 4 + 5 ))
${#variable}          #length of a variable
type -a <function>    #show us all Path possibilities - DON'T USE which, IT IS A RELIC
time <cmd>            #time the command 
```

### Filters (functions)

* `sort` - order rows
* `nl` - number lines
* `cut` - get columns from tabular-formatted file
* `sed` - search and replace, (stream editor)
* `uniq(ue)` - deduplicate adjacent lines
* `diff` - get differing lines of files
* `comm` - get common lines of files
* `cat` `(tac)` - print data (reverse order)

### Sequence of events

1. execute program - kernel loads its pre-written instructions (its code) 
2. begin process
3. setup "inherit" bash's file descriptors
  - FD2 error output
  - FD1 output
4. look for redirection

The beginning of the script should contain `#!/usr/bin/env bash`

* `#!` - hashbang tells the kernel what interpreter it needs
* `/usr/bin/env` - absolute pathname to any program that understands the language in your file and can take a single argument.  `/usr/bin/env` is a program that can find and start other programs.  
* `bash` - the one argument tells the program to find the bash program
* the script is run with a path: `./script`, so that bash is not looking for a command.  this is a security precaution, too, in-case virus was placed in file called `ls`. 

## Script

### Variables

* `x=10` - for a value
* lines=`cat $1 | wc -l` - for an output value
* `export Var` - every time a new process is created (to run another script or such) then make a copy of the variable and hand it over to the new process.

### Arguments within script

* `$0` - The name of the script.
* `$1 - $9` - Any command line arguments given to the script. $1 is the first argument, $2 the second and so on.
* `$#` - How many command line arguments were given to the script.
* `$*` - All of the command line arguments.

### Special variables within script

* `env` - all variables
* `$?` - exit status of the most recently run process.
* `$$` - process ID of the current script.
* `$USER` - username of the user running the script.
* `$HOSTNAME` - hostname of the machine the script is running on.
* `$SECONDS` - number of seconds since the script was started.
* `$RANDOM` - different random number each time is it called.
* `$LINENO` - current line number in the Bash script.

### Conditionals

IfThenElse

Compact method

```
true; echo $?; #or `echo $status` within fish

```

Block method

```
if [<test>]
then 
	<stmt>
	if [<test>]
	then exit
	fi
else 
	<stmt>
fi
```


## Bash commands

### Simple command 

syntax: `[ var=value ... ] cmd_name [ arg ... ] [ redirection ... ]`
    
* `[]` optional
* `...` repeated
* `[ var=value ... ]` variable assignments apply to the environment of this one command only
* `cmd_name` found in order of function, builtin, external program
* `arg` arguments
* `redirect` of input / output

### Syntax sugar

* pipe `|`  with `[time [-p]] [ ! ] command [ [|||&] command2 ... ]` - connect file descriptors to streams; standard output (FD 1) => standard input (FD 0).  standard error (FD 2) not included unless `|&` 
* list `command control-operator [ command2 control-operator ... ]` - separates commands, typically, just the newline
   - `;` start a new line, run commands in sequence
   - `||` run next command only if the command before it failed
* compound command `{...}` in `if list [ ;|<newline> ] then list [ ;|<newline> ] fi { list ; }`
* parallelize (coprocess) `coproc [ name ] command [ redirection ... ]` run in background
* function `name () compound-command [ redirection ]` no arguments in `()`

## Simple commands

### Literal

literal syntax: `'File 01'` (quotes), `File\ 01`(backslash), syntactic word-splitting `File 01`
    
```
echo "Good morning, $USER"    #variable expansion
 #Good morning, jason.beach
echo 'Good morning, $USER'    #prevent expansion
 #Good morning, $USER
 
rm -vr "/home/$username"    #correct
rm -vr /home/$username     #incorrect - interpreted as space, so all of home deleted
```

The golden rule on quoting is very simple:

* "If there is whitespace or a symbol in your argument, you must quote it.
* If there isn't, quotes are usually optional, but you can still quote it to be safe."

It is extremely rare for arguments to require no quoting, primarily inside [[ tests and around ${..+..} expansions. 


### Redirect
                                                                             
```
ls -l a b >myfiles.ls 2>/dev/null     #redirect FD1 to the file "myfiles.ls" and FD2 to the file "/dev/null"
ls -l a b >myfiles.ls 2>&1            #correct - send output and error bytes on the same stream
ls -l a b >myfiles.ls 2>myfiles.ls    #incorrect - never send two FDs to the same output
ls -l a b 2>&1 >myfiles.ls            #incorrect - sends FD2's output to the stream FD1 is connected to, which at the time is probably the terminal and not the file, because FD1 hasn't been redirected yet
ls -l a b &>myfiles.ls                #correct - convenience operator for writing FD1 and FD2            
```
                                                                             
The `/dev` directory is for device.  The `null` device is a special device that is always empty. Anything you write to it will be lost and nothing can be read from it. 

Use the `>&` operator to 'copy' file descriptors.  Syntax `2>&1` as "Make FD 2 write(>) to where FD(&) 1 is currently writing."
